In [1]:
import warnings

from sympy import true

warnings.filterwarnings("ignore")

In [2]:
from dotenv import load_dotenv
load_dotenv()

import os
gemini_api_key=os.getenv("GEMINI_API_KEY")
gemini_model_name=os.getenv("GEMINI_MODEL_NAME")
print(gemini_model_name)


gemini/gemini-2.5-flash


In [3]:
from crewai import LLM

llm = LLM(
    model=gemini_model_name,
    api_key=gemini_api_key,
    temperature=0.7
)
# response = llm.call(
#     "Analyze the following messages and return the name, age, and breed. "
#     "Meet Kona! She is 3 years old and is a black german shepherd."
# )
# print(response)

Creating a planner

In [4]:
from crewai import Agent, Task, Crew

In [5]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True,
    llm=llm
)

Agent : Writer

In [6]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

Agent:Editor

In [7]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

Task:Plan

In [8]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

Task: write

In [9]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

Task: Edit

In [10]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

creating crew:

In [11]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

running crew

In [12]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 123be09c-cd84-4564-80af-7e802749ed18                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.            │
│  2. Identify the target audience, considering their interests and pain points.                                  │
│  3. Develop a detailed content outline including an introduction, key points, and a call to action.             │
│  4. Include SEO keywords and relevant data or sources.                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Content Plan Document: Navigating the AI Revolution: Trends, Impact, and Future Outlook**                    │
│                                                                                                                 │
│  **1. Article Goal & Vision:**                                                                                  │
│  To provide a comprehensive and engaging overview of Artificial Intelligence, focusing on the latest trends,    │
│  key players, and its profound impact on business and society. The article aims to demystify complex AI         │
│  concepts, equip the audience with actionable insights, and foster informed decision-making regarding AI        │
│  adoption and its ethical implications.                                                                         │
│                                                                                                                 │
│  **2. Target Audience Analysis:**                                                                               │
│                                                                                                                 │
│  *   **Primary Audience:** Business Professionals, Decision-Makers, and Entrepreneurs.                          │
│      *   **Interests:** How AI can drive efficiency, reduce costs, create new revenue streams, gain             │
│  competitive advantage, automate processes, understand market disruptions, and future-proof their               │
│  organizations.                                                                                                 │
│      *   **Pain Points:** Overwhelmed by AI hype and jargon, fear of job displacement, ethical concerns (bias,  │
│  privacy), implementation challenges, justifying ROI, finding skilled AI talent, keeping up with rapid          │
│  technological changes.                                                                                         │
│  *   **Secondary Audience:** Tech Enthusiasts, Curious Individuals, and Aspiring Developers.                    │
│      *   **Interests:** Latest AI models and breakthroughs, technical advancements, ethical considerations,     │
│  career opportunities in AI, understanding AI's societal impact, open-source AI tools.                          │
│      *   **Pain Points:** Distinguishing hype from reality, understanding core AI concepts, navigating the      │
│  vast and rapidly evolving AI landscape, identifying reliable information sources.                              │
│                                                                                                                 │
│  **3. Prioritized Trends, Key Players, and Noteworthy News:**                                                   │
│                                                                                                                 │
│  *   **Latest Trends:**                                                                                         │
│      *   **Generative AI Dominance:** Large Language Models (LLMs), text-to-image/video generation, multimodal  │
│  AI. This is the most significant and transformative trend currently.                                           │
│      *   **Ethical AI & Regulation:** Increasing focus on responsible AI development, bias mitigation,          │
│  transparency, and global regulatory frameworks (e.g., 

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Writer                                                                                          │
│                                                                                                                 │
│  Task: 1. Use the content plan to craft a compelling blog post on Artificial Intelligence.                      │
│  2. Incorporate SEO keywords naturally.                                                                         │
│  3. Sections/Subtitles are properly named in an engaging manner.                                                │
│  4. Ensure the post is structured with an engaging introduction, insightful body, and a summarizing             │
│  conclusion.                                                                                                    │
│  5. Proofread for grammatical errors and alignment with the brand's voice.                                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: ddfcdd6f-7da5-48c6-af7d-f510cd510549                                                                     │
│  Agent: Content Planner                                                                                         │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: ddabebee-7af8-4760-9212-ede9116f3196                                                                     │
│  Agent: Content Writer                                                                                          │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```markdown                                                                                                    │
│  # Navigating the AI Revolution: Trends, Impact, and Your Path Forward                                          │
│                                                                                                                 │
│  Artificial Intelligence isn't just a buzzword; it's reshaping every facet of our lives at an unprecedented     │
│  pace. From automating routine tasks to generating breathtaking art and sophisticated code, AI technology is    │
│  no longer a futuristic concept but a present-day reality rapidly integrating into our daily existence. The     │
│  sheer scale and speed of its evolution mean that understanding AI's core principles and implications is no     │
│  longer optional for businesses and individuals, but a critical imperative.                                     │
│                                                                                                                 │
│  At its heart, Artificial Intelligence refers to the simulation of human intelligence in machines that are      │
│  programmed to think like humans and mimic their actions. This encompasses a broad spectrum of technologies,    │
│  including Machine Learning (ML), where systems learn from data without explicit programming, and Deep          │
│  Learning (DL), which uses neural networks inspired by the human brain. This article aims to cut through the    │
│  noise, providing a clear roadmap to understanding the latest AI trends, identifying the key players driving    │
│  innovation, dissecting its profound impact, and offering a forward-looking perspective for individuals and     │
│  businesses alike. We are at an inflection point where AI is moving from niche applications to mainstream       │
│  ubiquity, making informed engagement absolutely crucial for navigating the opportunities and challenges        │
│  ahead.                                                                                                         │
│                                                                                                                 │
│  ## The AI Landscape Today – Key Trends Shaping Our World                                                       │
│                                                                                                                 │
│  ### A. Generative AI's Explosion: Unleashing Creativity and Efficiency                                         │
│                                                                                                                 │
│  One of the most significant and transformative AI trends currently dominating the technological landscape is   │
│  the rise of Generative AI. This powerful subset of AI can create new content, ideas, solutions, and even       │
│  worlds, rather than just analyzing existing data. The advancements in Large Language Models (LLMs), such as    │
│  OpenAI's ChatGPT, Google's Gemini, and Anthropic's Claude, have fundamentally altered how we approach content  │
│  creation, coding, summarization, and customer service. These models can understand and generate human-like     │
│  text, proving invaluable for drafting reports, developing software, or powering sophisticated chatbots.        │
│                                                        

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 123be09c-cd84-4564-80af-7e802749ed18                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```markdown                                                                                      │
│  # Navigating the AI Revolution: Trends, Impact, and Your Path Forward                                          │
│                                                                                                                 │
│  Artificial Intelligence isn't just a buzzword; it's reshaping every facet of our lives at an unprecedented     │
│  pace. From automating routine tasks to generating breathtaking art and sophisticated code, AI technology is    │
│  no longer a futuristic concept but a present-day reality rapidly integrating into our daily existence. The     │
│  sheer scale and speed of its evolution mean that understanding AI's core principles and implications is no     │
│  longer optional for businesses and individuals, but a critical imperative.                                     │
│                                                                                                                 │
│  At its heart, Artificial Intelligence refers to the simulation of human intelligence in machines that are      │
│  programmed to think like humans and mimic their actions. This encompasses a broad spectrum of technologies,    │
│  including Machine Learning (ML), where systems learn from data without explicit programming, and Deep          │
│  Learning (DL), which uses neural networks inspired by the human brain. This article aims to cut through the    │
│  noise, providing a clear roadmap to understanding the latest AI trends, identifying the key players driving    │
│  innovation, dissecting its profound impact, and offering a forward-looking perspective for individuals and     │
│  businesses alike. We are at an inflection point where AI is moving from niche applications to mainstream       │
│  ubiquity, making informed engagement absolutely crucial for navigating the opportunities and challenges        │
│  ahead.                                                                                                         │
│                                                                                                                 │
│  ## The AI Landscape Today – Key Trends Shaping Our World                                                       │
│                                                                                                                 │
│  ### A. Generative AI's Explosion: Unleashing Creativity and Efficiency                                         │
│                                                                                                                 │
│  One of the most significant and transformative AI trends currently dominating the technological landscape is   │
│  the rise of Generative AI. This powerful subset of AI can create new content, ideas, solutions, and even       │
│  worlds, rather than just analyzing existing data. The advancements in Large Language Models (LLMs), such as    │
│  OpenAI's ChatGPT, Google's Gemini, and Anthropic's Claude, have fundamentally altered how we approach content  │
│  creation, coding, summarization, and customer service. These models can understand and generate human-like     │
│  text, proving invaluable for drafting reports, develo

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 3335dc24-6e26-43ef-abc3-fdfe47bddff6                                                                     │
│  Agent: Editor                                                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [13]:
# from IPython.display import Markdown
# Markdown(result.raw)

# from IPython.display import display, Markdown, Latex,
# display(Markdown(result.raw))
# # If you particularly want to display maths, this is more direct:
# display(Latex(result.raw))

In [17]:
from IPython.display import display_markdown
display_markdown(result, raw=True)
